In [1]:
import pickle

import torch
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

from image_colorization import *
from utils import *
from optim import *

%matplotlib inline

In [2]:
use_cuda = False
classification = True

In [3]:
image_list = get_images(n=10)

In [4]:
ab2cat = pickle.load(open('cache/ab2cat_10.pkl', 'rb'))
cat2ab = pickle.load(open('cache/cat2ab_10.pkl', 'rb'))
nearest_neighbors = pickle.load(open('cache/nearest_neighbors.pkl', 'rb'))
inv_distr = np.loadtxt('cache/class_weights.txt')

n_spaces = len(ab2cat)

In [5]:
preprocessor_class = lambda label: get_smoothed_label(label, nearest_neighbors, ab2cat)
preprocessor_reg = lambda label: np.clip(label, -128, 128) / 128

preprocessor = preprocessor_class if classification else preprocessor_reg
dset = ColorizationDataset(image_list, preprocessor)
loader = data.DataLoader(dset, shuffle=True, batch_size=4)

In [ ]:
n_outputs = n_spaces if classification else 2
model = ImageColorizer(n_outputs)
model.apply(weights_init)
optimizer = optim.RMSprop(model.parameters())

class_weights = torch.FloatTensor(inv_distr)

In [ ]:
num_epochs = 20
if classification:
    model = train_model_class(model, optimizer, loader, class_weights, num_epochs=num_epochs, show_every=None)
else:
    model = train_model_reg(model, optimizer, loader, num_epochs=num_epochs, show_every=None)

Epoch 0
Average loss: 0.00158353770773
Epoch 1
Average loss: 0.00125837221276
Epoch 2
Average loss: 0.0012836077949
Epoch 3
Average loss: 0.00100806832779
Epoch 4
Average loss: 0.00105463404907
Epoch 5
Average loss: 0.00104027297736
Epoch 6
Average loss: 0.00106163518891
Epoch 7
Average loss: 0.00098838932657
Epoch 8
Average loss: 0.00101318309316
Epoch 9
Average loss: 0.000899357449574
Epoch 10
Average loss: 0.00101495301351
Epoch 11


In [ ]:
if classification:
    predict_class(model, dset, cat2ab)
else:
    predict_reg(model, dset)